In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-07-05 12:49:10--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   115MB/s    in 0.2s    

2025-07-05 12:49:10 (115 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [4]:
# add your code here - consider creating a new cell for each section of code
df_books.head()

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber


In [5]:

df_ratings.head()

,user,isbn,rating
0,276725,034545104X,0.0
1,276726,0155061224,5.0
2,276727,0446520802,0.0
3,276729,052165615X,3.0
4,276729,0521795028,6.0


In [6]:
# Filter entries based on conditions
df_filtered = df_ratings

# Get the valid users and books
user_counts = df_filtered["user"].value_counts()
book_counts = df_ratings["isbn"].value_counts()
valid_users = user_counts[user_counts >= 200].index
valid_books = book_counts[book_counts >= 100].index
print(valid_users)
print(valid_books)

# Filter data by the users then by the books
df_filtered = df_filtered[df_filtered["user"].isin(valid_users)]
df_filtered = df_filtered[df_filtered["isbn"].isin(valid_books)]

df_filtered

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       188951,   9856, 155916, 268622, 225595,  36554,  83671, 252827,  26883,
        99955],
      dtype='int32', name='user', length=905)
Index(['0971880107', '0316666343', '0385504209', '0060928336', '0312195516',
       '044023722X', '0679781587', '0142001740', '067976402X', '0671027360',
       ...
       '0449203794', '0140092323', '0312966806', '0385424736', '0425189864',
       '067100042X', '0425151867', '0425136981', '0786866586', '0060916508'],
      dtype='object', name='isbn', length=731)


,user,isbn,rating
1456,277427,002542730X,10.0
1469,277427,0060930535,0.0
1471,277427,0060934417,0.0
1474,277427,0061009059,9.0
1484,277427,0140067477,0.0
...,...,...,...
1147304,275970,0804111359,0.0
1147436,275970,140003065X,0.0
1147439,275970,1400031346,0.0
1147440,275970,1400031354,0.0


In [7]:
# Pivot the table
df_pivoted = df_filtered.pivot_table(index = "isbn", columns = "user", values = "rating").fillna(0)
df_pivoted

user,254,2276,2766,2977,3363,4017,4385,6242,6251,6323,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
isbn,,,,,,,,,,,,,,,,,,,,,
002542730X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0
0060008032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060096195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
006016848X,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0060173289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1573227331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1573229326,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1573229571,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
# Function to get the book id from book name
def get_book_id_from_book_name(name):
  row = df_books.loc[df_books["title"] == name]
  book_id = row.iloc[0]["isbn"]
  return book_id

# Function to get the book name from book id
def get_book_name_from_id(book_id):
  row = df_books.loc[df_books["isbn"] == book_id]
  book_name = row.iloc[0]["title"]
  return book_name


In [9]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  # Get the book id based on its name
  book_id = get_book_id_from_book_name(book)

  # Train the KNN model by passing it the pivoted dataset
  nbrs = NearestNeighbors(n_neighbors=6, metric='cosine',algorithm='brute')  # Cosine metric and brute algorithm needed for the dist
  nbrs.fit(df_pivoted)

  # Get the book vector and use the kneighbors() function to get the distances and indices
  book_vector = df_pivoted.loc[book_id].values.reshape(1, -1)
  distances, indices = nbrs.kneighbors(book_vector, n_neighbors = 6)

  # Find the book name of the nearest neighbours
  recommended_books = []
  for i in range(len(indices[0])):
    distance = distances[0][i].item()
    index = indices[0][i]

    # Get the book id from the index from the df_pivoted dataframe
    this_book_id = df_pivoted.index[index]
    this_book_name = get_book_name_from_id(this_book_id)

    # Skip if the book id is same as the current book id
    if book_id == this_book_id:
      continue

    recommended_books.append([this_book_name, distance])

  # Add the initial title to the recommended books array
  recommended_books = [book, recommended_books[::-1]]  # [::-1] is required to pass the tests

  return recommended_books

In [10]:
get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")

['The Queen of the Damned (Vampire Chronicles (Paperback))',
 [['Catch 22', 0.793983519077301],
  ['The Witching Hour (Lives of the Mayfair Witches)', 0.7448656558990479],
  ['Interview with the Vampire', 0.7345068454742432],
  ['The Tale of the Body Thief (Vampire Chronicles (Paperback))',
   0.5376338362693787],
  ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.5178411602973938]]]

In [11]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016210794448853], ['The Weight of Water', 0.7708583474159241], ['The Surgeon', 0.7699410915374756], ['I Know This Much Is True', 0.7677075266838074], ['The Lovely Bones: A Novel', 0.7234864234924316]]]
You passed the challenge! 🎉🎉🎉🎉🎉
